<div align="right" style="text-align: right"><i>Peter Norvig<br>Aug 2020</i></div>

# War ([What is it Good For?](https://www.youtube.com/watch?v=bX7V6FAoTLc))

The [538 Riddler Classic for 28 August 2020](https://fivethirtyeight.com/features/can-you-cover-the-globe/) asks about the chance of winning the children's [card game **War**](https://en.wikipedia.org/wiki/War_%28card_game%29) in a **sweep**: a game where player **A** wins 26 turns in a row against player **B**. (In **War**, players are dealt 26 cards each and on  each turn they flip the top card in their respective hands; higher rank card wins. Other rules are not relevant to this problem.)

We'll analyze this problem and come up with a program to solve it; first let's get the imports out of the  way:

In [1]:
import random
import itertools
import collections
from statistics import mean

In my analysis I considered four different approaches to the problem, which I will describe in the order I considered them (although I actually implemented the final one first, and then came back to fill in the details of the earlier approaches).

# Approach 1: Simple Arithmetic?

A naive approach reasons as follows: there are 13 ranks, so perhaps the outcome probabilities for the first turn are:

     A wins=6/13; B wins=6/13; tie=1/13
     
And thus the probability that **A** wins 26 turns in a row would be:

In [2]:
(6/13) ** 26

1.8595392516568175e-09

or about 2 in a billion. Unfortunately, that reasoning is **wrong**, even for one turn. The probability of a tie (that is, both players flipping cards of the same rank) on the first turn is actually 3/51 (not 1/13 or 4/52) because after player **A** flips a card, there are 51 equiprobable cards remaining for player **B**, and 3 have the  same rank. So we actually have for the first turn:

    A wins=24/51; B wins=24/51; tie=3/51
    
And if every subsequent turn were the same, the probability that **A** wins 26 turns in a row would be:

In [3]:
(24/51) ** 26

3.0808297965386556e-09

But actually the probabilities on subsequent turns would be slightly different, depending on the cards picked in the previous turns. So simple arithmetic doesn't give us the answer.

# Approach 2: Brute Force Enumeration?

Brute force enumeration means:
- Consider every possible permutation of the deck of cards.
- For each permutation, deal out the cards and see whether or not **A** sweeps.
- The probability that **A** sweeps is the number of sweeps divided by the number of permutations.

Easy-peasy; here's the code. First the types:

In [4]:
Probability = float # Type: Probability is a number between 0.0 and 1.0
Deck = list         # Type: Deck is a list of card ranks

Now the functions:

In [5]:
def make_deck(ranks=13, suits=4) -> Deck: 
    """Make a list of ranks, each rank repeated `suits` times."""
    return list(range(ranks)) * suits

def A_sweeps(deck) -> bool: 
    """Upon dealing this deck, does player A win every turn?"""
    return all(deck[i] > deck[i + 1] for i in range(0, len(deck), 2))

def brute_force_p_sweep(deck) -> Probability:
    """The probability that A sweeps, considering every permutation of deck."""
    return mean(A_sweeps(d) for d in itertools.permutations(deck))

(Note that in Python the `bool` value `False` is equivalent to `0` and `True` is equivalent to `1`, so the `mean` of an iterable of `bool` values is the same as the proportion or probability of `True` values.)

Here we run the code on a tiny deck of just 8 cards, all with different ranks:

In [6]:
make_deck(8, 1)

[0, 1, 2, 3, 4, 5, 6, 7]

In [7]:
brute_force_p_sweep(make_deck(8, 1))

0.0625

There can be no ties, so this is four turns where on each turn **A** and **B** have equal chances of winning and the probability of **A** sweeping is: 

In [8]:
(1/2) ** 4

0.0625

If the 8-card deck had 4 ranks and 2 suits, then the probability of **A** sweeping would be less, because there could be ties:

In [9]:
make_deck(4, 2)

[0, 1, 2, 3, 0, 1, 2, 3]

In [10]:
brute_force_p_sweep(make_deck(4, 2))

0.03571428571428571

What about the real deck, with 52 cards? Unfortunately, there are 52! permutations (more than $10^{67}$), and even if we were clever about the duplicated ranks and the ordering of the 26 turns, and
even if we could process a billion deals a second, it would still take [millions of years](https://www.google.com/search?q=%2852%21+%2F+4%21%5E13+%2F+26%21%29+nanoseconds+in+years&oq=%2852%21+%2F+4%21%5E13+%2F+26%21%29+nanoseconds+in+years) to complete the brute force enumeration. And 538 Riddler wanted the answer by Monday.



# Approach 3: Simulation?

It would take too long to look at **all** the permutations, but we can  **randomly sample** the space of possible permutations; we call that a **simulation**:

In [11]:
def deals(deck, N) -> Deck: 
    """Yield N randomly shuffled deals of deck."""
    for _ in range(N):
        random.shuffle(deck)
        yield deck

def simulate(deck, N=10000) -> Probability:
    """Simulate N games of War, and return the estimated probability of A sweeping."""
    return mean(A_sweeps(d) for d in deals(deck, N))

In [12]:
simulate(make_deck())

0

Sampling 10,000 deals with the full 52-card deck we got zero sweeps. Since the probability of a sweep is probably somewhere around one in a billion, we  would need to look at trillions of deals to get a reliable estimate. That would require roughly a day of run time: much better than millions of years, but still not good enough.

# Approach 4: Abstract Incremental Enumeration!

The first three approaches didn't pan out. What next? 

It is feasible to consider every possible deal if we are careful. As discussed in my [How To Count Things](How%20To%20Count%20Things.ipynb) notebook, the idea is to represent a deck not as a concrete permutation of 52 cards but rather with a representation that is:
- **Abstract**: What really matters is not the exact rank of every card in the deck, but rather whether **A**'s next card is higher, lower, or the same as **B**'s next card. For example, if there are only two cards remaining and we know they have different ranks, then the probability of **A** winning is 1/2; it doesn't matter whether the cards are [10, 8] or [2, 5] or any of the 52 × 51 possibilities for two cards.
- **Incremental**: First we'll consider the possibilities for the two cards in the first turn, and only if **A** wins will we then move on to consider possible cards for the next turn. For cases where **A**  loses or ties the first turn, there is no need to consider the 50! permutations of the remaining cards.


The key to the abstract representation is knowing the cards of the same rank. I will define an abstract deck (or `AbDeck`) as a tuple where `deck[i]` gives the number of different ranks that have exactly `i` cards remaining in the deck. That's all the information you need to determine the probability of winning. (This representation is a bit wasteful, because `deck[0]` is always a redundant `0`, but the alternative would be to sprinkle the code with `[i - 1]` indexes, risking an off-by-one error).

Consider these sample abstract decks:

In [13]:
AbDeck = tuple # The Abstract Deck Type

deck = AbDeck((0, 0, 0, 0, 13))
tie1 = AbDeck((0, 0, 1, 0, 12))
dif1 = AbDeck((0, 0, 0, 2, 11))
end4 = AbDeck((0, 4))

- `deck` is the normal 52 card full deck with 13 ranks, each with 4 cards of that rank, so `deck[4]` is 13.
- `tie1` results when both players flip a card of the same rank on the first turn. <br>There are 12 ranks with 4 cards each and 1 rank with 2 cards.
- `dif1` results when the players flip cards of different ranks on the first turn.  <br>There  are 11 ranks each with 4 cards and 2 ranks with 3 cards. 
- `end4` is a deck you would see near the end of a game, with just 4 remaining cards, each of a different rank. 

Note that `tie1` and `dif1` are the only two possible abstract deck outcomes for turn 1. That's a lot better than the 52! possibilities that we would need  to consider on turn 1 with a concrete deck. 

We can create abstract decks and count the number of cards as follows:

In [14]:
def make_abdeck(ranks=13, suits=4) -> AbDeck: 
    """Make an abstract deck."""
    return (0,) * suits + (ranks,)
    
def ncards(deck: AbDeck) -> int:
    """Number of cards remaining in an abstract deck."""
    return sum(i * deck[i] for i in range(len(deck)))

In [15]:
[ncards(d) for d in (deck, tie1, dif1, end4)]

[52, 50, 50, 4]

# Probability Distributions

We want to keep track of the probability of **A** winning, over a variety of possible events (flips of cards). We'll do that with the help of a class called `PDist` (or **probability distribution**), a mapping that lists all the possible abstract decks that might occur on a turn, each mapped to their probability of occurrance.

In [16]:
class PDist(collections.Counter): 
    """A probability distribution of {abstract_deck: probability}."""

# Abstract Incremental Enumeration Strategy

Now we're ready to outline a strategy to exactly and efficiently compute `p_sweep(deck)`, the probability that **A** sweeps a game of **War**:

- Start with `P` being a **probability distribution** of deck outcomes after 0 turns: `{deck: 1.0}`.
- **for** each of the 26 turns (or in general ncards(deck) / 2 turns):
  - Update `P` to be the result of playing a turn, which is given by:
    - **for** each of the entries in `P`:
      - See what possible outcomes can arise from selecting 2 cards from the deck
      - Update the new `PDist` `P1`  to include each outcome where **A** might sweep, with appropriate probability
- Sum the probabilities in `P`

In [17]:
def p_sweep(deck: AbDeck) -> Probability:
    """The probability that player A sweeps a game of War."""
    P = PDist({deck: 1.0}) # The initial probability distribution
    for turn in range(ncards(deck) // 2):
        P = play_turn(P)
    return sum(P.values())

def play_turn(P) -> PDist:
    """Play one turn with all possible card choices for players A and B; return
    the probability distribution of outcomes where A still might sweep."""
    P1 = PDist() # The probability distribution after this turn
    for deck in P:
        for deck2, p_Awin in select2(deck):
            P1[deck2] += P[deck] * p_Awin
    return P1

Now the key is figuring out:
- All the possible ways to select two cards from the deck.
- The probability of each selection.
- Who won (or was it a tie) with that selection.

In [18]:
def select1(deck) -> list:
    """All ways to pick one card from deck, returning a list of tuples:
    (remaining deck, probability of pick, index of pick in deck)"""
    return [(remove(i, deck), i * deck[i] / ncards(deck), i)
           for i in range(len(deck)) if deck[i]]

def select2(deck) -> list:
    """All ways to select two cards from deck, returning a list of tuples:
    (remaining deck, probability that deck occurs and player A wins)."""
    return [(deck2, p_i * p_j * p_Awin(deck1, i, j))
            for deck1, p_i, i in select1(deck)
            for deck2, p_j, j in select1(deck1)
            if p_Awin(deck1, i, j) > 0]
  
def p_Awin(deck1, i, j) -> Probability:
    """Probability that A wins turn when A selects i giving deck1 and B selects j."""
    p_tie = 1 / deck1[j] if j == i - 1 else 0
    return (1 - p_tie) / 2

def remove(i, deck) -> AbDeck:
    """Remove one card from deck[i]."""
    deck1 = list(deck)
    deck1[i] -= 1
    if i - 1 != 0:
        deck1[i - 1] += 1
    return AbDeck(deck1)

# The Answer!

What's the probability that player **A** will win in a sweep?

In [19]:
p_sweep(deck)

3.132436174322294e-09

The probability that **A** sweeps a game of **War** is a little over 3 in a billion. 

(By the way, this computation took less than 1/10 second, a big improvement over millions of years.) 

That's the answer to *my* question about the probability of **A** sweeping, but 538 Riddler actually posed the question somewhat differently: "*How many games of War would you expect to play until you had a game that lasted just 26 turns with no wars, like Duane’s friend’s granddaughter?*" That is, they want to know the inverse of the probability, and they are allowing for either **A** or **B** to sweep. So that would be:

In [20]:
1 / (2 * p_sweep(deck))

159620171.7049113

 We would expect a sweep about once every 160 million games. I have to say, I'm begining to doubt Duane’s friend’s granddaughter's claim. 

# Working through the algorithm

Let's work through how `p_sweep`, `play_turn`, `select1` and `select2` work. We'll start by reminding ourself what the starting `deck` is, and then `select1` card from it:

In [21]:
deck

(0, 0, 0, 0, 13)

In [22]:
select1(deck)

[((0, 0, 0, 1, 12), 1.0, 4)]

This is saying with probability 1.0 the result of selecting one card is a deck where there are 12 ranks with four-of-a-kind and 1 rank with three-of-a-kind. The selected card came from index 4 in the deck (it was one of a four-of-a-kind).

Now we'll try selecting two cards:

In [23]:
select2(deck)

[((0, 0, 0, 2, 11), 0.47058823529411764)]

This says that the only result of the first turn in which **A** wins has 11 ranks with four-of-a-kind and 2 ranks with 3-of-a-kind. The probability of this outcome is  0.47058823529411764, which, as we calculated earlier, is 24/51. The rest of the probability goes to an equiprobable result in which **B** wins, and to `(0, 0, 1, 0, 12)`, which indicates a tie on the first turn. Since these outcomes don't result in **A** winning, they do not appear in the result from `select2`.

Now let's work through how `p_sweep` repeatedly calls `play_turn`, updating the `PDist` `P`, which is initially:

In [24]:
P = PDist({deck: 1.0})
P

PDist({(0, 0, 0, 0, 13): 1.0})

Here's the outcome of playing the first turn:

In [25]:
P = play_turn(P)
P

PDist({(0, 0, 0, 2, 11): 0.47058823529411764})

Now for the second turn:

In [26]:
P = play_turn(P)
P

PDist({(0, 0, 2, 0, 11): 0.0017286914765906362,
       (0, 0, 1, 2, 10): 0.05070828331332534,
       (0, 0, 0, 4, 9): 0.16902761104441777})

And the third turn:

In [27]:
P = play_turn(P)
P

PDist({(0, 2, 0, 0, 11): 3.0650558095578654e-06,
       (0, 1, 1, 1, 10): 0.00040458736686163827,
       (0, 0, 2, 2, 9): 0.010114684171540957,
       (0, 1, 0, 3, 9): 0.0017981660749406148,
       (0, 0, 3, 0, 10): 0.00020229368343081916,
       (0, 0, 1, 4, 8): 0.048550484023396595,
       (0, 0, 0, 6, 7): 0.04315598579857475})

We'll leave it as an exercise for the reader to work through these, but one thing you can clearly see is that the total probability of winning is becoming smaller with every turn:

In [28]:
sum(P.values())

0.10422926617455494

There is only a 10% chance of winning three turns in a row. This will steadily fall to 3-in-a-billion after 26 turns. 

# Gaining Confidence in the Answer

The answer should be somewhere close to the arithmetic calculation of $(24/51)^{26}$; in fact we see that it differs by less than 2%:

In [29]:
p_sweep(deck) / ((24/51) ** 26)

1.0167508045532467

Although the brute force algorithm can't handle a 52-card deck, we can use it to verify that there is no difference between `brute_force_p_sweep(deck)` and `p_sweep(deck)` for small decks; this gives us more confidence that both are correct (or maybe both have similar errors):

In [30]:
for ranks, suits in [(2, 1), (4, 1), (6, 1), (8, 1),
                     (2, 2), (2, 3), (3, 2), (2, 4), (4, 2)]:
    p1 = brute_force_p_sweep(make_deck(ranks, suits))
    p2 =             p_sweep(make_abdeck(ranks, suits))
    print(f'deck({ranks}, {suits}):  p = {p1:.5f};  diff = {abs(p1 - p2):.9f}')

deck(2, 1):  p = 0.50000;  diff = 0.000000000
deck(4, 1):  p = 0.25000;  diff = 0.000000000
deck(6, 1):  p = 0.12500;  diff = 0.000000000
deck(8, 1):  p = 0.06250;  diff = 0.000000000
deck(2, 2):  p = 0.16667;  diff = 0.000000000
deck(2, 3):  p = 0.05000;  diff = 0.000000000
deck(3, 2):  p = 0.06667;  diff = 0.000000000
deck(2, 4):  p = 0.01429;  diff = 0.000000000
deck(4, 2):  p = 0.03571;  diff = 0.000000000


And we can do some unit tests on components (we should do more):

In [31]:
def test() -> bool:
    # If there are `2N` cards in a deck, all with different ranks, 
    # then the probability that A sweeps is `(1/2) ** N`.
    for N in range(200):
        assert p_sweep((0, 2 * N)) == (1/2) ** N
    # Remove a single card from abstract deck
    assert remove(4, (0, 0, 0, 0, 13)) == (0, 0, 0, 1, 12)
    assert remove(4, (0, 0, 0, 1, 12)) == (0, 0, 0, 2, 11)
    assert remove(3, (0, 0, 0, 1, 12)) == (0, 0, 1, 0, 12)
    assert remove(1, (0, 2)) == (0, 1)
    assert remove(1, (0, 1)) == (0, 0)
    # Count cards in abstract deck
    assert [ncards(d) for d in (deck, tie1, dif1, end4)] == [52, 50, 50, 4]
    # Select cards
    assert select1((0, 0, 0, 0, 13)) == [((0, 0, 0, 1, 12), 1.0, 4)]
    assert select2((0, 0, 0, 0, 13)) == [((0, 0, 0, 2, 11), 0.47058823529411764)]
    assert select1((0, 2)) == [((0, 1), 1.0, 1)]
    assert select2((0, 2)) == [((0, 0), 0.5)]
    return True
    
test()

True